# Collisions Dataset Analysis

In [4]:
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import collisionfuncs
import warnings
pd.set_option('display.float_format','{:.2f}'.format)
warnings.filterwarnings('ignore')

## Exploratory Analysis & Data Cleaning

In [5]:
collisions = pd.read_csv("../matched_data/crash_matched_data/collisions_matched.tsv", sep = '\t')

In [6]:
# filter out false LATITUDE and LONGITUDE 
criterion1 = (collisions['LATITUDE'] > 38) & (collisions['LATITUDE'] < 42) & (collisions['LONGITUDE'] < -72) & (collisions['LONGITUDE'] > -76)
# filter entries with matched streetID
criterion2 = collisions['streetID'].notnull()
collisions = collisions[criterion1 & criterion2]

In [4]:
collisions.head()

collision_id  CRASH DATE CRASH TIME   BOROUGH ZIP CODE  LATITUDE  \
16            22  12/13/2021       6:30    QUEENS 11372.00     40.75   
17            22  12/13/2021       6:30    QUEENS 11372.00     40.75   
23            28  12/14/2021      17:15  BROOKLYN 11211.00     40.71   
28            39  07/09/2021       0:43       NaN      NaN     40.72   
30            41  04/24/2022       4:49       NaN      NaN     40.86   

    LONGITUDE                 LOCATION ON STREET NAME CROSS STREET NAME  ...  \
16     -73.89    (40.75373, -73.88505)      82 STREET         34 AVENUE  ...   
17     -73.89    (40.75373, -73.88505)      82 STREET         34 AVENUE  ...   
23     -73.95  (40.710957, -73.951126)   GRAND STREET      UNION AVENUE  ...   
28     -73.89   (40.720535, -73.88885)   ELIOT AVENUE               NaN  ...   
30     -73.87  (40.855972, -73.869896)    BOSTON ROAD   BRONX PARK EAST  ...   

                               int2   zip1     zip2                   ll1  \
16          82+STREET+%26+34+AVENUE  11370 11372.00  40.755620,-73.885380   
17      82ND+STREET+%26+34TH+AVENUE  11370 11372.00  40.755620,-73.885380   
23  GRAND+STREET+%26+LORIMER+STREET  11211 11211.00  40.710968,-73.951115   
28         ELIOT+AVENUE+%26+69+LANE  11378 11379.00  40.720162,-73.889666   
30   BOSTON+ROAD+%26+PELHAM+PARKWAY  10462 10467.00  40.855815,-73.869924   

                     ll2  streetID  lat1  long1  lat2  long2  
16  40.753699,-73.885026    357.00 40.76 -73.89 40.75 -73.89  
17  40.753699,-73.885026    358.00 40.76 -73.89 40.75 -73.89  
23  40.711175,-73.948909   1298.00 40.71 -73.95 40.71 -73.95  
28  40.720531,-73.888847   1119.00 40.72 -73.89 40.72 -73.89  
30  40.857248,-73.868940    626.00 40.86 -73.87 40.86 -73.87  

[5 rows x 46 columns]

In [5]:
collisions.columns

Index(['collision_id', 'CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE',
       'LATITUDE', 'LONGITUDE', 'LOCATION', 'ON STREET NAME',
       'CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5',
       'coords', 'index_right', 'Roadway.Name', 'From', 'To', 'int1', 'int2',
       'zip1', 'zip2', 'll1', 'll2', 'streetID', 'lat1', 'long1', 'lat2',
       'long2'],
      dtype='object')

In [6]:
collisions.loc[:, ['NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED']].describe()

NUMBER OF PERSONS INJURED  NUMBER OF PERSONS KILLED  \
count                  221458.00                 221458.00   
mean                        0.31                      0.00   
std                         0.69                      0.04   
min                         0.00                      0.00   
25%                         0.00                      0.00   
50%                         0.00                      0.00   
75%                         0.00                      0.00   
max                        17.00                      5.00   

       NUMBER OF PEDESTRIANS INJURED  NUMBER OF PEDESTRIANS KILLED  \
count                      221460.00                     221460.00   
mean                            0.07                          0.00   
std                             0.26                          0.03   
min                             0.00                          0.00   
25%                             0.00                          0.00   
50%                             0.00                          0.00   
75%                             0.00                          0.00   
max                             6.00                          2.00   

       NUMBER OF CYCLIST INJURED  NUMBER OF CYCLIST KILLED  \
count                  221460.00                 221460.00   
mean                        0.04                      0.00   
std                         0.19                      0.01   
min                         0.00                      0.00   
25%                         0.00                      0.00   
50%                         0.00                      0.00   
75%                         0.00                      0.00   
max                         3.00                      1.00   

       NUMBER OF MOTORIST INJURED  NUMBER OF MOTORIST KILLED  
count                   221460.00                  221460.00  
mean                         0.21                       0.00  
std                          0.64                       0.03  
min                          0.00                       0.00  
25%                          0.00                       0.00  
50%                          0.00                       0.00  
75%                          0.00                       0.00  
max                         17.00                       5.00

In [7]:
boroughs = gpd.read_file(gplt.datasets.get_path('nyc_boroughs'))

## Number of Persons Injured
+ The average number of persons injured or killed over all collisions on a specific street segment.
+ The average number of person injured and the average number of motorist injured are lower in Mahantten.

In [18]:
analysis = ['NUMBER OF PERSONS INJURED',
       'NUMBER OF MOTORIST INJURED']
for column in analysis:
    collisionfuncs.pointplot(collisions, column)

## Future Analysis
+ How different collision factors contributed to these collisions
    + Time: before and after COVID.
+ Join the dataset with speed, traffic
+ How collisions changed before and after bumps were setup
+ Risk of collision: collision/traffic amount.
